In [4]:
import numpy as np
import os
import pickle
import csv

# array of id of record, file name of each ECG
record_ids = []
# array where each element is signal for ECG file
records = []

for file in os.listdir('./ECGDataDenoised'):
    a = file.split('.')
    if a[1] != 'csv':
        # skip non csv files
        continue
    # read signal csv file
    rec_orig = []
    with open('./ECGDataDenoised/' + file) as f:
        reader = csv.reader(f)
        for row in reader:
            rec_orig.append(row[0])
    f.close()
    rec_500_hz = []
    if len(rec_orig) == 5000:
        # resample to 100 Hz
        rec_500_hz = scipy.signal.resample(rec_orig, 1000)
    record_ids.append(a[0])
    records.append(rec_500_hz)

with open('record_ids.pkl', 'wb') as f:
    pickle.dump(record_ids,f)
    f.close()

with open('records.pkl', 'wb') as f:
    pickle.dump(records,f)
    f.close()
    
print('Done :)')

Done :)


In [8]:
records[9]

[[...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],


In [9]:
record_dict = {}

for i in range(0,len(record_ids)):
    record_dict[record_ids[i]] = i

# atrial flutter ECGs
AF_files = []
# sinus irregularity ECGs
SA_files = []
# supraventricular tachycarida ECGs
SVT_files = []
# atrial tachycardia ECGs
AT_files = []

# read annotations from annotation csv file
with open('Annotations.csv', 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rec_id = row[0]
        rec_ann = row[1]
        rec_data = np.array([[float(s)] for s in records[record_dict[rec_id]]])
        if len(rec_data) != 5000 | (rec_ann == 'AVNRT') | (rec_ann == 'AVRT') | (rec_ann == 'SAAWR'):
        # get rid of samples we don't understand :)
            print('Skipped sample', rec_id, 'with annotation', rec_ann, 'and length', len(rec_data))
        # GET ATRIAL FLUTTER FILES
        elif rec_ann == 'AF':
            AF_files.append(rec_data)
        # GET SINUS IRREGULARITY FILES
        elif rec_ann == 'SA':
            SA_files.append(rec_data)
        # GET SUPRAVENTRICULAR TACHYCARDIA FILES
        elif rec_ann == 'SVT':
            SVT_files.append(rec_data)
        # GET ATRIAL TACHYCARDIA FILES
        elif rec_ann == 'AT':
            AT_files.append(rec_data)   
    csvfile.close()
        
print('We have', len(AF_files), 'atrial flutter samples')
print('We have', len(SA_files), 'sinus irregularity samples')
print('We have', len(SVT_files), 'supraventricular tachycardia samples')
print('We have', len(AT_files), 'atrial tachycardia samples')

with open('AF_files.pkl', 'wb') as f:
    pickle.dump(AF_files, f)
    f.close()

with open('SA_files.pkl', 'wb') as f:
    pickle.dump(SA_files, f)
    f.close()

with open('SVT_files.pkl', 'wb') as f:
    pickle.dump(SVT_files, f)
    f.close()

with open('AT_files.pkl', 'wb') as f:
    pickle.dump(AT_files, f)
    f.close()

TypeError: float() argument must be a string or a number, not 'list'

In [10]:
import pickle
import csv

# shorten AF ECG files to 8 second (4096) ECG, rather than 10 second (5000)
AF_files_4096 = []

with open('AF_files.pkl', 'rb') as f:
    AF_files_4096 = pickle.load(f)
    f.close()

for i in range(0, len(AF_files_4096)):
    AF_files_4096[i] = AF_files_4096[i][:4096]

In [17]:
import numpy as np

a_mins = []
a_maxs = []
for i in range(0, len(AF_files_4096)):
    a_mins.append(np.min([a[0] for a in AF_files_4096[i]]))
    a_maxs.append(np.max([a[0] for a in AF_files_4096[i]]))
print('min is', min(a_mins), ', max is', max(a_maxs))

min is -3648.6 , max is 2283.7


In [24]:
# convert each 4096-len ECG to a 2d "image"

AF_images = []
for i in range(0, len(AF_files_4096)):
    AF_image = []
    for p in range(0, 4096, 64):
        AF_image.append([a/4096.0 for a in AF_files_4096[i][p:p+64]])
    AF_images.append(AF_image)

AF_images[0][0]

[array([-0.00441089]),
 array([-0.00610718]),
 array([-0.00751123]),
 array([-0.00833984]),
 array([-0.00840015]),
 array([-0.00768677]),
 array([-0.00646021]),
 array([-0.00521094]),
 array([-0.00447656]),
 array([-0.00462793]),
 array([-0.00573364]),
 array([-0.00711719]),
 array([-0.00725342]),
 array([-0.00735449]),
 array([-0.00740308]),
 array([-0.00739355]),
 array([-0.00733325]),
 array([-0.00723804]),
 array([-0.00712402]),
 array([-0.0070022]),
 array([-0.00687915]),
 array([-0.00676025]),
 array([-0.00664868]),
 array([-0.00654297]),
 array([-0.00643262]),
 array([-0.0062981]),
 array([-0.00611743]),
 array([-0.00587476]),
 array([-0.00556299]),
 array([-0.00517554]),
 array([-0.0047002]),
 array([-0.00412134]),
 array([-0.00343262]),
 array([-0.0026377]),
 array([-0.00174353]),
 array([-0.00075667]),
 array([0.000322]),
 array([0.00148484]),
 array([0.0027041]),
 array([0.00394189]),
 array([0.00517407]),
 array([0.00640405]),
 array([0.00765088]),
 array([0.00892236]),
 ar

In [ ]:
# to convert 1D signal to 2D gray-scale image
import matplotlib

for count, i in enumerate(array):
  fig = plt.figure(frameon=False)
  plt.plot(i) 
  plt.xticks([]), plt.yticks([])
  for spine in plt.gca().spines.values():
     spine.set_visible(False)

  filename = directory + '/' + str(count)+'.png'
  fig.savefig(filename)
  im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
  im_gray = cv2.resize(im_gray, (64, 64), interpolation = cv2.INTER_LANCZOS4)
  cv2.imwrite(filename, im_gray)